<a href="https://colab.research.google.com/github/Anas-art-source/AI_experiments/blob/main/PDF_STRUCTURED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install pdfquery PyPDF2 pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 25.7 MB/s eta 0:00:00


In [4]:
from pdfquery import PDFQuery

pdf = PDFQuery('/content/invoice.pdf.pdf')
pdf.load()
pdf.doc.catalog

{'Type': /'Catalog',
 'Version': /'1.4',
 'Pages': <PDFObjRef:2>,
 'StructTreeRoot': <PDFObjRef:3>,
 'MarkInfo': <PDFObjRef:4>,
 'Lang': b'en',
 'ViewerPreferences': <PDFObjRef:5>}

In [5]:
# x = pdf.pq(':contains("Invoicing templates")')
# print(x)

In [6]:
from lxml import etree


tree = pdf.get_tree(0)

root =tree.getroot()
for element in root:
    # Do something with each element, for example:
    print(etree.tostring(element))

b'<LTPage y0="0" y1="842.25" x0="0" x1="595.5" width="595.5" height="842.25" bbox="[0, 0, 595.5, 842.25]" pageid="2" rotate="0" page_index="0" page_label=""><LTRect y0="0.0" y1="842.25" x0="0.0" x1="595.5" width="595.5" height="842.25" bbox="[0.0, 0.0, 595.5, 842.25]" linewidth="0" pts="[[0.0, 842.25], [595.5, 842.25], [595.5, 0.0], [0.0, 0.0]]"><LTRect y0="0.0" y1="842.25" x0="0.0" x1="595.5" width="595.5" height="842.25" bbox="[0.0, 0.0, 595.5, 842.25]" linewidth="0" pts="[[0.0, 842.25], [595.5, 842.25], [595.5, 0.0], [0.0, 0.0]]"><LTTextBoxHorizontal y0="736.192" y1="779.19" x0="59.528" x1="281.884" width="222.357" height="42.997" bbox="[59.528, 736.192, 281.884, 779.19]" index="0"><LTTextLineHorizontal y0="760.192" y1="779.19" x0="59.528" x1="157.475" width="97.948" height="18.997" bbox="[59.528, 760.192, 157.475, 779.19]" word_margin="0.1">G I N Y A R D </LTTextLineHorizontal><LTTextLineHorizontal y0="736.192" y1="755.19" x0="59.528" x1="281.884" width="222.357" height="18.997" bb

In [7]:
data  = pdf.extract([
       ('with_formatter', 'text'),
      ('Company_first_name', ':in_bbox("59.528, 760.192, 157.475, 779.19")'),
      ('Company_second_name', ':in_bbox("59.528, 736.192, 281.884, 755.19")'),
      ('job_name', ':in_bbox("478.482, 770.151, 535.735, 782.151")'),
      ('invoice_number', ':in_bbox("461.221, 755.151, 538.8, 767.151")'),
      ('date', ':in_bbox("488.443, 740.151, 535.735, 752.151")'),
      ('to_name', ':in_bbox("59.528, 623.193, 139.794, 634.188")'),
      ('to_address_1', ':in_bbox("59.528, 603.822, 159.443, 615.822")'),
      ('to_address_2', ':in_bbox("59.528, 587.322, 159.884, 599.322")'),
      ('from_name', ':in_bbox("323.216, 623.193, 483.229, 634.188")'),
      ('from_address_1', ':in_bbox("323.216, 603.822, 423.132, 615.822")'),
      ('from_address_2', ':in_bbox("59.528, 587.322, 159.884, 599.322")'),
      ('sub_total', ':in_bbox("462.752, 304.918, 504.414, 315.913")'),
      ('tax', ':in_bbox("462.752, 289.918, 497.082, 300.913")'),
      ('total', ':in_bbox("462.752, 274.918, 502.952, 285.913")'),
      ('remainder', ':in_bbox("409.397, 229.868, 508.516, 240.863")'),
      ('total_pay_now', ':in_bbox(" 409.397, 229.868, 508.516, 240.863")'),

 ])

In [8]:
from lxml import etree

# Assuming pdf.get_tree(0) returns the XML tree
tree = pdf.get_tree(0)

root = tree.getroot()

# Define an XPath expression to select LTTextBoxHorizontal elements
xpath_expr = ".//LTTextLineHorizontal"
i = 0

product_list = []
# Iterate over elements matching the XPath expression
for element in root.xpath(xpath_expr):
    # Extract text content of the element
    for el in element:
      i += 1
      if i >4:
        text_content = el.text
        product_list.append(text_content)


product_list = product_list[:-8]
dump = []
for idx in range(0,len(product_list)):
  if idx % 5 == 0:
    product_name = product_list[idx]
    product_quantity = product_list[idx+1]
    tax = product_list[idx+2]
    price = product_list[idx+3]
    subtotal = product_list[idx+4]
    dump.append({
        'product_name': product_name,
        'quantity': product_quantity,
        'tax': tax,
        "price": price,
        'subtotal': subtotal
        })


In [9]:
data['product_list'] = dump

In [10]:
data

{'Company_first_name': 'G I N Y A R D',
 'Company_second_name': 'I N T E R N A T I O N A L C O .',
 'job_name': 'Job Name',
 'invoice_number': 'Invoice 01234',
 'date': '3/05/22',
 'to_name': 'J u l i a n a S i l v a J u l i a n a S i l v a',
 'to_address_1': '123 Anywhere St.,',
 'to_address_2': 'Any City, ST 12345',
 'from_name': 'G i n y a r d I n t e r n a t i o n a l C o . G i n y a r d I n t e r n a t i o n a l C o .',
 'from_address_1': '123 Anywhere St.,',
 'from_address_2': 'Any City, ST 12345',
 'sub_total': '$300.00',
 'tax': '$45.00',
 'total': '$345.00',
 'remainder': '$345.00 PAY N OW $345.00 PAY N OW',
 'total_pay_now': '$345.00 PAY N OW $345.00 PAY N OW',
 'product_list': [{'product_name': 'Design ',
   'quantity': '1 ',
   'tax': '15% ',
   'price': '$100.00 ',
   'subtotal': '$115.00 '},
  {'product_name': 'Artwork ',
   'quantity': '1 ',
   'tax': '15% ',
   'price': '$100 ',
   'subtotal': '$115.00 '},
  {'product_name': 'Social Media ',
   'quantity': '1 ',
   'tax

In [11]:
def extract_pdf(filename: str):
  pdf = PDFQuery(filename)
  pdf.load()
  data  = pdf.extract([
       ('with_formatter', 'text'),
      ('Company_first_name', ':in_bbox("59.528, 760.192, 157.475, 779.19")'),
      ('Company_second_name', ':in_bbox("59.528, 736.192, 281.884, 755.19")'),
      ('job_name', ':in_bbox("478.482, 770.151, 535.735, 782.151")'),
      ('invoice_number', ':in_bbox("461.221, 755.151, 538.8, 767.151")'),
      ('date', ':in_bbox("488.443, 740.151, 535.735, 752.151")'),
      ('to_name', ':in_bbox("59.528, 623.193, 139.794, 634.188")'),
      ('to_address_1', ':in_bbox("59.528, 603.822, 159.443, 615.822")'),
      ('to_address_2', ':in_bbox("59.528, 587.322, 159.884, 599.322")'),
      ('from_name', ':in_bbox("323.216, 623.193, 483.229, 634.188")'),
      ('from_address_1', ':in_bbox("323.216, 603.822, 423.132, 615.822")'),
      ('from_address_2', ':in_bbox("59.528, 587.322, 159.884, 599.322")'),
      ('sub_total', ':in_bbox("462.752, 304.918, 504.414, 315.913")'),
      ('tax', ':in_bbox("462.752, 289.918, 497.082, 300.913")'),
      ('total', ':in_bbox("462.752, 274.918, 502.952, 285.913")'),
      ('remainder', ':in_bbox("409.397, 229.868, 508.516, 240.863")'),
      ('total_pay_now', ':in_bbox(" 409.397, 229.868, 508.516, 240.863")'), ])


  tree = pdf.get_tree(0)
  root = tree.getroot()

  # Define an XPath expression to select LTTextBoxHorizontal elements
  xpath_expr = ".//LTTextLineHorizontal"
  i = 0

  product_list = []
  # Iterate over elements matching the XPath expression
  for element in root.xpath(xpath_expr):
      #Extract text content of the element
      for el in element:
        i += 1
        if i >4:
          text_content = el.text
          product_list.append(text_content)


  product_list = product_list[:-8]
  dump = []
  for idx in range(0,len(product_list)):
    if idx % 5 == 0:
      product_name = product_list[idx]
      product_quantity = product_list[idx+1]
      tax = product_list[idx+2]
      price = product_list[idx+3]
      subtotal = product_list[idx+4]
      dump.append({
            'product_name': product_name,
            'quantity': product_quantity,
            'tax': tax,
            "price": price,
            'subtotal': subtotal
            })


  data['product_list'] = dump

  return data

In [12]:
extract_pdf('/content/invoice.pdf.pdf')

{'Company_first_name': 'G I N Y A R D',
 'Company_second_name': 'I N T E R N A T I O N A L C O .',
 'job_name': 'Job Name',
 'invoice_number': 'Invoice 01234',
 'date': '3/05/22',
 'to_name': 'J u l i a n a S i l v a J u l i a n a S i l v a',
 'to_address_1': '123 Anywhere St.,',
 'to_address_2': 'Any City, ST 12345',
 'from_name': 'G i n y a r d I n t e r n a t i o n a l C o . G i n y a r d I n t e r n a t i o n a l C o .',
 'from_address_1': '123 Anywhere St.,',
 'from_address_2': 'Any City, ST 12345',
 'sub_total': '$300.00',
 'tax': '$45.00',
 'total': '$345.00',
 'remainder': '$345.00 PAY N OW $345.00 PAY N OW',
 'total_pay_now': '$345.00 PAY N OW $345.00 PAY N OW',
 'product_list': [{'product_name': 'Design ',
   'quantity': '1 ',
   'tax': '15% ',
   'price': '$100.00 ',
   'subtotal': '$115.00 '},
  {'product_name': 'Artwork ',
   'quantity': '1 ',
   'tax': '15% ',
   'price': '$100 ',
   'subtotal': '$115.00 '},
  {'product_name': 'Social Media ',
   'quantity': '1 ',
   'tax

In [13]:
from pdfquery import PDFQuery

pdf = PDFQuery('/content/judgement_2.pdf')
pdf.load()
pdf.doc.catalog

{'Pages': <PDFObjRef:2>, 'Type': /'Catalog'}

In [14]:
from lxml import etree


tree = pdf.get_tree(0)

root =tree.getroot()
for element in root:
    # Do something with each element, for example:
    print(etree.tostring(element))

b'<LTPage y0="0" y1="841.92" x0="0" x1="595.32" width="595.32" height="841.92" bbox="[0, 0, 595.32, 841.92]" pageid="31" rotate="0" page_index="0" page_label=""><LTTextLineHorizontal y0="788.508" y1="800.508" x0="385.32" x1="520.771" width="135.451" height="12.0" bbox="[385.32, 788.508, 520.771, 800.508]" word_margin="0.1"><LTTextBoxHorizontal y0="788.508" y1="800.508" x0="385.32" x1="520.771" width="135.451" height="12.0" bbox="[385.32, 788.508, 520.771, 800.508]" index="0">Case No: 3312598/2022 </LTTextBoxHorizontal></LTTextLineHorizontal><LTTextLineHorizontal y0="685.896" y1="709.897" x0="143.88" x1="465.148" width="321.268" height="24.001" bbox="[143.88, 685.896, 465.148, 709.897]" word_margin="0.1"><LTTextBoxHorizontal y0="685.896" y1="709.897" x0="143.88" x1="465.148" width="321.268" height="24.001" bbox="[143.88, 685.896, 465.148, 709.897]" index="1">EMPLOYMENT TRIBUNALS </LTTextBoxHorizontal></LTTextLineHorizontal><LTTextLineHorizontal y0="645.708" y1="657.708" x0="93.746" x1="

In [15]:
# Assuming pdf.get_tree(0) returns the XML tree
tree = pdf.get_tree(0)

root = tree.getroot()

# Define an XPath expression to select LTTextBoxHorizontal elements
xpath_expr = ".//LTTextBoxHorizontal"

# Iterate over elements matching the XPath expression
for element in root.xpath(xpath_expr):
    # Extract text content of the element
    text_content = element.text

    # Print or process the text content as needed
    print(text_content)

Case No: 3312598/2022 
EMPLOYMENT TRIBUNALS 
laimant: 
 Mark Sturgess 
espondent: Cambridge Country Club Ltd 
eard at: 
 Watford by video 
 On: 9 & 10 November 2023 
efore: 
 Employment Judge K Hunt 
epresentation 

 Miss Webber 
RESERVED JUDGMENT 
1. 
The complaint of unfair dismissal is well-founded. The claimant was 
unfairly dismissed. 
2. 

3. 

4. 
The complaint of wrongful dismissal in breach of contract in relation to 
notice pay is well-founded. 
5. 

6. 

REASONS 
10.5 Reserved judgment with reasons – rule 62 
March 2017 


In [16]:
pdf.extract([
      ('with_formatter', 'text'),
      ('year', 'LTTextLineHorizontal:contains("£434.75")', lambda match: match.text())
 ])

{'year': ''}

In [17]:
import PyPDF2
# Open the PDF file
t = ''
with open('/content/judgement_2.pdf', 'rb') as file:
    # Create a PDF reader object
    reader = PyPDF2.PdfReader(file)
    # Iterate over each page in the PDF
    for page_num in range(len(reader.pages)):
        # Get the current page
        page = reader.pages[page_num]

        # Extract text from the page
        text = page.extract_text()

        # Split the text into lines
        lines = text.split('\n')

        # Iterate over each line in the page
        for line in lines:
            # Print or process the line as needed
            t =  t + line +'\n'
            # print(text)
            # print(line)

t

'Case No: 3312598/2022 \n10.5 Reserved judgment with reasons – rule 62  March 2017 \n \n \nEMPLOYMENT TRIBUNALS \n \n \nClaimant:   Mark Sturgess \n \nRespondent:  Cambridge Country Club Ltd  \n \n \nHeard at:    Watford by video     On: 9 & 10 November 2023   \n \nBefore:    Employment Judge K Hunt    \n \nRepresentation \nClaimant:    Miss Webber   \nRespondent:   Mr Hine   \n \n \nRESERVED JUDGMENT \n1. The complaint of unfair dismissal is well-founded. The claimant was \nunfairly dismissed. \n \n2. The respondent unreasonably failed to comply with the ACAS Code of \nPractice on Disciplinary and Grievance Procedures 2015 and it is just and \nequitable to increase the compensatory award payable to the claimant by \n20% in accordance with s 207A Trade Union & Labour Relations \n(Consolidation) Act 1992 \n \n3. The claimant caused or contributed to the dismissal by blameworthy \nconduct and it is just and equitable to reduce the basic and compensatory \naward payable to the claimant by

In [18]:
print(t)

Case No: 3312598/2022 
10.5 Reserved judgment with reasons – rule 62  March 2017 
 
 
EMPLOYMENT TRIBUNALS 
 
 
Claimant:   Mark Sturgess 
 
Respondent:  Cambridge Country Club Ltd  
 
 
Heard at:    Watford by video     On: 9 & 10 November 2023   
 
Before:    Employment Judge K Hunt    
 
Representation 
Claimant:    Miss Webber   
Respondent:   Mr Hine   
 
 
RESERVED JUDGMENT 
1. The complaint of unfair dismissal is well-founded. The claimant was 
unfairly dismissed. 
 
2. The respondent unreasonably failed to comply with the ACAS Code of 
Practice on Disciplinary and Grievance Procedures 2015 and it is just and 
equitable to increase the compensatory award payable to the claimant by 
20% in accordance with s 207A Trade Union & Labour Relations 
(Consolidation) Act 1992 
 
3. The claimant caused or contributed to the dismissal by blameworthy 
conduct and it is just and equitable to reduce the basic and compensatory 
award payable to the claimant by 40%. 
 
4. The complaint of wrong

In [31]:
from pdfquery import PDFQuery

pdf = PDFQuery('/content/SOF_MV LILA SEOUL SOF_667916 (1).pdf')
pdf.load()
pdf.doc.catalog

{'Type': /'Catalog', 'Pages': <PDFObjRef:12>}

In [33]:
from lxml import etree


tree = pdf.get_tree(0)

root =tree.getroot()
for element in root:
    # Do something with each element, for example:
    print(etree.tostring(element))

b'<LTPage y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" pageid="5" rotate="0" page_index="0" page_label=""><LTImage y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" name="lm1" colorspace="[&quot;/DeviceRGB&quot;]" bits="8" imagemask="null" srcsize="[1240, 1753]" stream="&lt;PDFStream(3): raw=1182651, {\'Type\': /\'XObject\', \'Subtype\': /\'Image\', \'Length\': 1182651, \'Filter\': /\'FlateDecode\', \'ColorSpace\': /\'DeviceRGB\', \'BitsPerComponent\': 8, \'Width\': 1240, \'Height\': 1753}&gt;"><LTFigure y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" name="lm1" matrix="[595, 0, 0, 841, 0, 0]"/></LTImage></LTPage>'


In [32]:
# Assuming pdf.get_tree(0) returns the XML tree
tree = pdf.get_tree(0)

root = tree.getroot()

# Define an XPath expression to select LTTextBoxHorizontal elements
xpath_expr = ".//LTTextBoxHorizontal"

# Iterate over elements matching the XPath expression
for element in root.xpath(xpath_expr):
    # Extract text content of the element
    element.
    text_content = element.text

    # Print or process the text content as needed
    print(text_content)

In [38]:
from lxml import etree
import fitz

def display_element_info(element):
    if element.tag == "LTImage":
        image_info = element.attrib
        print("Image:")
        print(image_info)
    else:
        print(etree.tostring(element))

def extract_images_from_pdf(pdf_file):
    pdf_document = fitz.open(pdf_file)
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        for image in page.get_images(full=True):
            yield image[0]

tree = pdf.get_tree(0)
root = tree.getroot()

for element in root:
    display_element_info(element)

pdf_file = "/content/SOF_MV LILA SEOUL SOF_667916 (1).pdf"
for image in extract_images_from_pdf(pdf_file):
    print("Image:", image)

b'<LTPage y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" pageid="6" rotate="0" page_index="0" page_label=""><LTImage y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" name="lm1" colorspace="[&quot;/DeviceRGB&quot;]" bits="8" imagemask="null" srcsize="[1240, 1753]" stream="&lt;PDFStream(3): raw=1182651, {\'Type\': /\'XObject\', \'Subtype\': /\'Image\', \'Length\': 1182651, \'Filter\': /\'FlateDecode\', \'ColorSpace\': /\'DeviceRGB\', \'BitsPerComponent\': 8, \'Width\': 1240, \'Height\': 1753}&gt;"><LTFigure y0="0" y1="841" x0="0" x1="595" width="595" height="841" bbox="[0, 0, 595, 841]" name="lm1" matrix="[595, 0, 0, 841, 0, 0]"/></LTImage></LTPage>'
Image: 3
Image: 6
Image: 9
Image: 3
Image: 6
Image: 9
Image: 3
Image: 6
Image: 9


In [39]:
import fitz  # PyMuPDF
import os

def extract_images_from_pdf(pdf_file):
    pdf_document = fitz.open(pdf_file)
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        for index, img in enumerate(page.get_images(full=True)):
            image_bytes = pdf_document.extract_image(img[0])
            image_name = f"page{page_num+1}_image{index}.png"
            with open(image_name, "wb") as f:
                f.write(image_bytes["image"])

pdf_file = "/content/SOF_MV LILA SEOUL SOF_667916 (1).pdf"
extract_images_from_pdf(pdf_file)
